In [1]:
from hello import hello, hello3
from officelib.xllib import *
from officelib import const

In [2]:
HERR = (hello.HelloError, hello3.HelloError)
NLI = (hello.HelloError, hello3.NotLoggedInError)

from time import sleep
def set_mg(sp):
    call(app.setmg, 1, sp)        
        
def call(f, *args):
    
    while True:
        try:
            rv = f(*args)
        except NLI:
            app.login()
        except requests.exceptions.ConnectTimeout as e:
            print(str(e))
            print("Make sure you're on the right network!")
        else:
            return rv
        
        
def update_ss(cell, val):
    cell.Value = val
    c = cell.Offset(2, 1)
    c.Select()
    return c
        
def frange(start, stop, step):
    i = 0
    v = start + i * step
    while v < stop:
        yield v
        i += 1
        v = start + i * step
        

def run_test(name, mfc_max, c=None):
    tmin = mfc_max / 10
    print("Testing ", name, "MFC: %.1f LPM Max" % mfc_max)
    app.login()
    for fr in frange(tmin, mfc_max+tmin/2, tmin):
        set_mg(fr)
        v = input("%.2f LPM:" % fr)
        if v.lower() in ("break", "quit", "q", "stop"):
            break
        c = update_ss(c, v)

def air_on(v):
    app.setph(2,0,0)
    app.setdo(2,0,0)
    
def n2_on(v):
    app.setph(2,0,0)
    app.setdo(1, 100, 0)
    
def o2_on(v):
    app.setph(2,0,0)
    app.setdo(1, 0, 100)
    
def co2_on(v):
    app.setconfig("pH", "CO2_Manual_Max_(%)", 100)
    app.setph(1, 100, 0)
    app.setdo(2, 0, 0)
    app.setmg(1, v)
    input("Purging line of air. Press enter when done.")
    

cc = {
    "co2": 4,
    "air": 1,
    "n2": 2,
    "o2": 3
}

on_fn = {
    "co2": co2_on,
    "air": air_on,
    "n2": n2_on,
    "o2": o2_on
}

    
def s():
    return xl.Selection

def run(name, mfcm):
    c = cc[name]
    app.login()
    if name == "co2":
        co2m = app.getConfig()['pH']['CO2_Manual_Max_(%)']
    on_fn[name](mfcm)
    run_test(name, mfcm, cells(2,c))
    if name == "co2":
        app.login()
        app.setconfig("pH", "CO2_Manual_Max_(%)", co2m)

def all_off():
    app.login()
    app.setag(0, 12)
    app.setpumpa(0, 0)
    app.setpumpb(0, 0)
    app.setpumpc(0, 0)
    app.setpumpsample(0, 0)
    
def connect():
    global app
    v = input("Version: ").lower()
    ip = input("Enter IP Address: ")
    if v == "3":
        app = hello3.HelloApp(ip)
    else:
        app = hello.HelloApp(ip)
    app.getConfig = types.MethodType(getConfig, app)
    
def init():
    global xl, wb, ws, cells, cell_range
    xl, wb, ws, cells = xlObjs()
    cell_range = cells.Range
    cell_range("A1:D1").Value = [("Air", "N2", "O2", "CO2")]
    

In [3]:
from hello.hello3 import HelloXML
def getConfig(self):
    query = "?&call=getConfig"
    rsp = self.send_request(query)
    xml = HelloXML(rsp)
    self._verify_xml(xml)
    # James renamed the clusters to SV_Air
    # and SV_Mag to make it easier for him
    # to keep track of. The "proper" solution
    # would be to just return the proper XML 
    # element when parsing. The easiest solution
    # is to just check for the key, since conversion
    # from XML to dict occurs upstream. 
    try:
        return xml.data['System_Variables_Mag']
    except KeyError:
        return xml.data['System_Variables']
import types

In [4]:
init()
connect()
all_off()

Version: 3
Enter IP Address: 192.168.1.12


In [5]:
run("air", 0.5)

Testing  air MFC: 0.5 LPM Max
0.05 LPM:53.2
0.10 LPM:103.3
0.15 LPM:153.4
0.20 LPM:204.1
0.25 LPM:254.6
0.30 LPM:305.4
0.35 LPM:354.7
0.40 LPM:403.6
0.45 LPM:454
0.50 LPM:504.3


In [7]:
run("n2", 0.5)

Testing  n2 MFC: 0.5 LPM Max
0.05 LPM:53.3
0.10 LPM:102.8
0.15 LPM:153.4
0.20 LPM:203.8
0.25 LPM:254.9
0.30 LPM:306.4
0.35 LPM:356.3
0.40 LPM:406.6
0.45 LPM:4556.2
0.50 LPM:505.4


In [8]:
run("o2", .5)

Testing  o2 MFC: 0.5 LPM Max
0.05 LPM:53.6
0.10 LPM:103.1
0.15 LPM:154.2
0.20 LPM:204.3
0.25 LPM:255.9
0.30 LPM:308.7
0.35 LPM:359.7
0.40 LPM:409.6
0.45 LPM:460.5
0.50 LPM:510.9


In [9]:
run("co2", .1)

Purging line of air. Press enter when done.
Testing  co2 MFC: 0.1 LPM Max
0.01 LPM:29.4
0.02 LPM:29.4
0.03 LPM:29.4
0.04 LPM:40.0
0.05 LPM:50.7
0.06 LPM:61.3
0.07 LPM:71.6
0.08 LPM:81.8
0.09 LPM:92.2
0.10 LPM:102.6


In [10]:
all_off()